# Create frames for training pretrained collision detection model

In [ ]:
#read video from path: 
import cv2

import pandas as pd

import os


In [ ]:
oracle_time = 4
train_path = os.listdir('nexar-collision-prediction/train')
train_vids = [os.path.join('/nexar-collision-prediction/train', vid) for vid in train_path if '_' not in vid]
train_df = pd.read_csv('datasets/Nexus_Data/train.csv')

In [ ]:
train_vids

In [ ]:
#get train id from the video path
train_df.columns

# Find the cars in the image

In [ ]:
def convert_seconds_to_frame(seconds, fps = 30):
    return int(seconds * fps)

def convert_frame_to_seconds(frame, fps = 30):
    return frame / fps

### Create Jsons for accident data

In [ ]:
save_train_accident_path = 'datasets/Nexus_Data/train/accident'
save_train_no_accident_path = 'datasets/Nexus_Data/train/no_accident'

In [ ]:
for train_vid in train_vids:
    
    train_id = int(train_vid.split('/')[-1].split('.')[0])
    print('vid name', train_vid)
    df_id = train_df[train_df['id'] == train_id]

    clipStart = df_id['time_of_alert'].values[0] #- oracle_time
    if clipStart < 0:
        clipStart = 0
    clipEnd = df_id['time_of_event'].values[0] + oracle_time

    cap = cv2.VideoCapture(train_vid)
    fr_number = 0
    #draw bounding box of car in last frame
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == False:
            break
        second = convert_frame_to_seconds(fr_number)
        print(second)

        if second < clipStart:
            #save frame as image
            print(f'{save_train_no_accident_path}/{train_id}_{fr_number}.jpg')
            cv2.imwrite(f'{save_train_no_accident_path}/{train_id}_{fr_number}.jpg', frame)
            fr_number += 1
            continue
        elif second > clipEnd:
            break
        else: 
            #save frame as image
            print(f'{save_train_accident_path}/{train_id}_{fr_number}.jpg')
            cv2.imwrite(f'{save_train_accident_path}/{train_id}_{fr_number}.jpg', frame)
            fr_number += 1


